In [1]:
import numpy as np # Importing numpy for performing array and matrix operations 
import pandas as pd # Importing pandas to read data in dataframe

# Libraries to plot the distribution of classes 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime #I will use this to Convert ISO8601 format to unix time
from matplotlib.pyplot import figure

import os
import random

import time #I will use this to Convert ISO8601 format to unix time
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from sklearn import linear_model  #Importing linear models like logistics regression and SVC
from sklearn.preprocessing import LabelEncoder  # To convert every class  into numeric labels 
from sklearn.metrics import f1_score       # to calculate F1 score 
from sklearn.metrics import classification_report # to know every metric
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras import backend as K
from collections import deque
from tensorflow.keras.layers import Layer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input,concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model, backend as K

In [2]:
data1= pd.read_csv('/content/drive/MyDrive/AAIC_Assignment/ytrain_NpxebDC.csv', parse_dates=['timestamp'],
index_col= ['timestamp']) # Reading the csv file in data dataframe 

In [3]:
for col in data1:    # for every column in the data frame
    if col!='timestamp':
        data1[col]=data1[col].fillna('Offline')  # Filling the missing class label with Down as explained above
data1['S7-T1'].isnull().sum() # counting the NAN values

0

In [4]:
data, test_data=train_test_split(data1, test_size=0.10, shuffle=False)

In [5]:
def date_time(data):
# Initializing Empty lists
   
    month=[]
    week=[]
    date=[]
    hr=[]
    minute=[]

# I have made timestamp column as an index of the dataframe for easy plotting operations
    for i in data.index: # For every index of the dataframe
    #print(i)
    #break
    #dt = datetime.datetime.strptime(i, "%Y-%m-%dT%H:%M:%S%z")
        dt=i
    # Appending the month, day, weekday, hour and minute in lists
        
        month.append(dt.month)
        week.append(dt.weekday()) #Monday is 0 and Sunday is 6
        date.append(dt.day)
        hr.append(dt.hour)
        minute.append(dt.minute)
# making X as a data frame 
    return pd.DataFrame(list(zip( month, week, date, hr, minute)), columns =['Month_name', 'Week_number', 'Date', 'Hour', 'Minute'])

In [6]:
X=date_time(data)
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [7]:
X_test=date_time(test_data)
X_test.head()

,Month_name,Week_number,Date,Hour,Minute
0,10,4,2,4,45
1,10,4,2,5,0
2,10,4,2,5,15
3,10,4,2,5,30
4,10,4,2,5,45


In [8]:
### DEFINE T2V LAYER ###
#https://towardsdatascience.com/time2vec-for-time-series-features-encoding-a03a4f3f937e
class T2V(tf.keras.layers.Layer):
    
    def __init__(self, output_dim=None):
        super().__init__()
        self.output_dim = output_dim
        
        
    def build(self, input_shape):

        self.W = self.add_weight(name='W',shape=(input_shape[-1], self.output_dim), initializer='uniform',trainable=True)

        self.P = self.add_weight(name='P',shape=(input_shape[1], self.output_dim), initializer='uniform',trainable=True)

        self.w = self.add_weight(name='w', shape=(input_shape[1], 1),initializer='uniform', trainable=True)

        self.p = self.add_weight(name='p',shape=(input_shape[1], 1),initializer='uniform', trainable=True)

        #super(T2V, self).build(input_shape)
        
    def call(self, x):
        
        original = tf.multiply(self.w, x)
        original=tf.add(original, self.p)
        
        sin_trans = tf.sin(K.dot(x, self.W) + self.P)
        
        return tf.concat([sin_trans, original], -1)

In [9]:
def label_encoder(column):
    '''
    This function help us to Encode the class labels of 
    the corresponding terminal into their respective 
    numeric labels
    '''
    if column =='Available': # Available ==0
        return 0
    elif column=='Charging': # charging==1
        return 1
    elif column=='Passive': # Passive==2
        return 2
    elif column=='Offline': # offline==3
        return 3
    else: return 4    # Down==4
# utilize it along with apply method

In [10]:
### DEFINE MODEL STRUCTURES ###

def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def T2V_NN(t2v_dim, dim):
    
    set_seed(33)
    
    inp = layers.Input(shape=(dim,1))
    x = T2V(t2v_dim)(inp)
    x = layers.LSTM(50, activation='relu')(x)
    #x= Dense(100, activation='relu')(x)
    #x=tf.keras.layers.BatchNormalization()(x)
    #x=Dense(64, activation='relu')(x)
    x=layers.Dropout(0.2)(x)
    #x=Dense(10)(x)
    x = layers.Dense(5,activation='softmax')(x)
    

    m = Model(inp, x)

    return m

    

In [11]:
class_weight = {0: 0.5,
                1: 4.5,
               2: 5.1,
               3:3.1,
               4:3.2}

In [12]:
tf.executing_eagerly()
warnings.filterwarnings("ignore")
f1_state_terminal=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
f1_state_terminal_test=[0,0,0,0,0, 0] # order available, passive, charging, offline, down
label_count=[0,0,0,0,0,0]
for i in tqdm(range(0,data.shape[1])):
 
  tf.keras.backend.clear_session()
  terminal= data.columns[i] # Extracting the column names from the dataframe
    #num_class=data[terminal].unique() # Unique labels present in the dataframe column
  weekend=data[terminal].apply(label_encoder)
  num_class=weekend.unique()
  num_class.sort()
  weekend_test=test_data[terminal].apply(label_encoder)
  num_class_test=weekend_test.unique()
  num_class_test.sort()
  if len(num_class)>1:
    unit=len(num_class)
    #print(unit)
    y=np.array(weekend).reshape(-1,1) # label encoding class labels
    y_test=np.array(weekend_test).reshape(-1,1)
    y_train1 = to_categorical(y, 5)
    y_test1 = to_categorical(y_test, 5)
    #print(y_train1.shape)
    #print(y_test1.shape)   
            # define model
    model = T2V_NN(32, 5)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

    
    #class_weights = class_weight.compute_class_weight('balanced',weekend.unique(y) ,data[terminal])
    #class_weights = dict(enumerate(class_weights))
    model.fit(X, y_train1, epochs=20, batch_size=100, verbose=0, class_weight=class_weight)
        
    y_pred=model.predict(X)
    val_predict=np.argmax(y_pred, axis=1) 
    val_target=np.argmax(y_train1, axis=1) 
    state_f1=f1_score(val_target, val_predict, average=None)
        
    y_pred_test=model.predict(X_test)
    val_predict=np.argmax(y_pred_test, axis=1) 
    val_target=np.argmax(y_test1, axis=1) 
    state_f1_test=f1_score(val_target, val_predict, average=None)
        
    idx=0
    for label in num_class:
            #print("more than 1",num_class)
        f1_state_terminal[label]+=state_f1[idx]
        label_count[label]+=1
        idx+=1
            #print('passed from if ')
            
    idx=0
    for label in num_class_test:
        f1_state_terminal_test[label]+=state_f1_test[idx]
        idx+=1
          
  else:
    for label in num_class:
            #print('1 class',num_class)
        f1_state_terminal[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
        label_count[label]+=1
            #print('passed from else')
        
    for label in num_class_test:
        f1_state_terminal_test[label]+=1 # Adding F1 score as 1 to the state where a terminal is in only one mode
 


  0%|          | 0/273 [00:00<?, ?it/s]

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


100%|██████████| 273/273 [2:07:28<00:00, 28.02s/it]


In [13]:
av_count=0
ch_count=0
pa_count=0
do_count=0
of_count=0
for i in tqdm(range(0,data.shape[1])):
    
    terminal= data.columns[i] # Extracting the column names from the dataframe
    labels=data[terminal]
    for i in labels:
        #print(i)
        if i=='Available':
            av_count+=1
        if i=='Charging':
            ch_count+=1
        if i=='Passive':
            pa_count+=1
        if i=='Down':
            do_count+=1
        if i=='Offline':
            of_count+=1
tot_count=av_count+ch_count+pa_count+do_count+of_count
print(tot_count)
prob_list=[0,0,0,0,0]# order available, passive, charging, offline, down
prob_list[0]=av_count/tot_count
prob_list[1]=pa_count/tot_count
prob_list[2]=ch_count/tot_count
prob_list[3]=of_count/tot_count
prob_list[4]=do_count/tot_count

100%|██████████| 273/273 [00:03<00:00, 85.10it/s]

7673484


In [14]:
#print('The counts of the class labels in train data', label_count)
#print("The Combined F1 scores of Terminals ",f1_state_terminal) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal[i]/=273
print("The Combined Average F1 scores of train Terminals ",f1_state_terminal) # order available, passive, charging, offline, down

#print("The Combined F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline, down
for i in range(len(label_count)-1):
    f1_state_terminal_test[i]/=273
print("The Combined Average F1 scores of test Terminals ",f1_state_terminal_test) # order available, passive, charging, offline,down

f1_final=0
#print(" The probabilities of the train states",prob_list)
for i in range(len(prob_list)):
    f1_final+=prob_list[i]*f1_state_terminal[i]
print('The final F1 score is', f1_final)

f1_final_test=0

for i in range(len(prob_list)):
    f1_final_test+=prob_list[i]*f1_state_terminal_test[i]
print('The final F1 score is', f1_final_test)

The Combined Average F1 scores of train Terminals  [0.6242982686601043, 0.28245953905003734, 0.4487842805130965, 0.6678907756612773, 0.34569488591457675, 0]
The Combined Average F1 scores of test Terminals  [0.32994522241882873, 0.11866553748905556, 0.09062716694879726, 0.23224475265440925, 0.0627496136916367, 0]
The final F1 score is 0.5732008284082488
The final F1 score is 0.25913652987586877
